In [26]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import pickle
import tools
import subprocess
from collections import defaultdict
import whisper
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
#inputVideo = "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker092/speaker092_0002.mp4"
inputVideo = r"inputs/dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
videoFrames = int(videoDuration*25)
fps = 25
#tools.splitVideo(inputVideo,2,videoDuration)

In [5]:
#Carga vídeo
res, facePos, faceFrames = tools.saveMultiFace(inputVideo,detector,50)
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR
print("Nº CARAS:",len(res))
for cara in res.keys():
    print(len(res[cara]))


Nº CARAS: 3
244
244
3


In [6]:
print(faceFrames[2])
# cv2.imshow("My Video", res[3][50])
# cv2.waitKey(0)

[24, 45, 46]


In [7]:
model = talkNet()
#model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))
windowSize = 21

06-17 13:03:47 Model para number = 15.01


In [8]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(4309)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [9]:
# correctFrames, totalFrames = 0, 0
# correctSamples, totalSamples = 0, 0
# totalPreds = []
# for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
#     with torch.no_grad():    
#         output = model((audioFeature,visualFeature))
#         labels = labels.cuda()
#         batchPreds = torch.reshape(output[1], labels.shape)
#         #Precision a nivel de video
#         videoPreds = torch.mode(batchPreds,dim=1)[0]
#         totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
#         labelMode = torch.mode(labels,dim=1)[0]
#         correctSamples += (videoPreds == labelMode).sum().float()
#         totalSamples += len(labels)
#         # Precision a nivel de frame
#         labels = labels.reshape((-1))
#         correctFrames += (output[1] == labels).sum().float()
#         totalFrames += len(labels) 
# #7865
# print("Frame acc:", correctFrames/totalFrames)
# print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res[0])).unsqueeze(0)
iVideo = tools.padVideo(res[0],center,51).unsqueeze(0)
# print(iAudio[40],item[0][40])
# print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
# print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
# print(audio[0],item[0][0])

c:\Users\jmmol\Desktop\COSAS V7\TFM\tools.py:182: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  video = torch.FloatTensor(video)


In [10]:
totalScores = defaultdict(list)
meanWSize = 5
meanSideSize = int((meanWSize-1)/2)
# MEDIA FRAMES
# for actualSpeaker in res.keys():
#     print("SPEAKER", actualSpeaker)
#     for i in range(len(res[actualSpeaker])):
#         #center = i
#         center = faceFrames[actualSpeaker][i]
#         print("FRAME Nº ",center)
#         iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
#         iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
#         #print("audio shape: ", audio.shape)
#         #print("video shape: ", video.shape)
#         # iAudio = item[0].unsqueeze(0)
#         # iVideo = item[1].unsqueeze(0)
#         print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
#         scores,labels= model((iAudio,iVideo))
#         #print(output)
#         print("Pred ->",torch.mode(labels)[0],"\tProb Pos ->",torch.mean(scores))
#         predArray[actualSpeaker].append(torch.mean(labels).detach().cpu().numpy())


# SECUENCIAL
sideWindowSize = int((windowSize-1)/2)
print("sidewindow:",sideWindowSize)
for actualSpeaker in res.keys():
    print("SPEAKER", actualSpeaker)
    for i in range(0,len(res[actualSpeaker])+windowSize,windowSize):
        #center = i
        center = faceFrames[actualSpeaker][0]+i
        #print("FRAME Nº ",center)
        iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
        iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
        #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
        scores,labels= model((iAudio,iVideo))
        totalScores[actualSpeaker].extend(scores[:,1].detach().cpu().numpy().tolist())
        #predArray[actualSpeaker].extend(labels.detach().cpu().numpy().tolist())

    # MEAN SLIDING WINDOW
    for i in range(len(totalScores[actualSpeaker])):
        ini = max(0,i-meanSideSize) # No negative values
        end = i+meanSideSize+1 # +1 as python does not take into account last value
        #print(ini,end)
        totalScores[actualSpeaker][i] = np.mean(totalScores[actualSpeaker][ini:end])
    print(totalScores[actualSpeaker])

predArray = defaultdict(list)
thr = 0.5
# APPLY THRESHOLD
for actualSpeaker in res.keys():
    # Delete < 0 frame predictions (initial)
    totalScores[actualSpeaker] = totalScores[actualSpeaker][sideWindowSize:len(res[actualSpeaker])+sideWindowSize]
    # Classify as speaking if score higher than threshold
    for sc in totalScores[actualSpeaker]:
        predLabel = 1 if sc > thr else 0
        predArray[actualSpeaker].append(predLabel)

    print(len(predArray[actualSpeaker]))

sidewindow: 10
SPEAKER 0
[0.004799175153796871, 0.004579463934836288, 0.0037739154494677977, 0.0038161831100781756, 0.0037051982286696638, 0.003936192759809395, 0.009205893911023935, 0.010414992852715155, 0.012577283986321589, 0.01117215189543744, 0.011490531284855863, 0.012090785480056378, 0.011347806560896085, 0.012376232178889994, 0.01337804154103707, 0.014918865792421576, 0.01608786860365176, 0.01615965004018011, 0.015131540873665741, 0.040210313425765386, 0.0814299696272264, 0.16317931774951439, 0.17759904235512797, 0.1719961929989024, 0.1215474355352569, 0.11600195302178089, 0.11630409392701806, 0.11227170798499753, 0.12846669675172687, 0.12370025623754796, 0.11491283551887166, 0.11539040810853567, 0.1310288013677315, 0.15201809321561552, 0.1233728440231567, 0.10965281860229163, 0.11888618916762202, 0.14400788304545306, 0.1354489968375964, 0.11864920182805032, 0.08370643207996448, 0.06732358195801602, 0.04046821366099394, 0.03256422807279806, 0.02611484978318504, 0.02985256111020

In [11]:
def createVideo(videoName,imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/videos/{videoName}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

# Returns a list of [start,end] timestamps in seconds of the parts where a speaker is speaking with minimum length of minLength frames
def getSpeaking(arr,minLength,fps):
    prev_idx = 0
    posFrames = 0
    idx_list = []
    for i,num in enumerate(arr):
        if num == 1:
            posFrames +=1
        else:
            if i-prev_idx >= minLength:
                idx_list.append((prev_idx/fps,i/fps))
            prev_idx = i
            posFrames = 0
    return idx_list

def saveFullVideo(videoName):
    cap = cv2.VideoCapture(inputVideo)
    videoImages = []
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while True:
        ret, image = cap.read()
        videoImages.append(image)
        if ret == False:
            break


    for speakerN in totalScores.keys():
        for i, fr in enumerate(faceFrames[speakerN]): #Para cada frame en el que aparezca la cara
            try:
                image = videoImages[fr]
                greenValue = int(255*predArray[speakerN][i])
                redValue = 255-greenValue
                color = (0,greenValue,redValue)
                #print(color)
                xmin,ymin,xmax,ymax = facePos[speakerN][i]
                image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
                cv2.putText(image, "{:.2f}".format(totalScores[speakerN][i]*100), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
            except:
                pass
    cap.release()
    cv2.destroyAllWindows()
    
    
    createVideo(videoName,videoImages,audioPath,width,height)

In [12]:
#Asignación de bounding boxes y probabilidades al video de output
os.makedirs("outputs/videos", exist_ok=True)
path = os.path.normpath(inputVideo)
videoName = str(path.split(os.sep)[-1])
saveFullVideo(videoName)

#print(predArray[3])

b''


In [32]:
# GUARDAR DATOS 
# COGER SOLO FRAGMENTO DE AUDIO EN EL QUE SE DETECTA HABLANTE
os.makedirs("outputs/pickles", exist_ok=True)
model = whisper.load_model("small")
transcription = model.transcribe(audioPath, language="es", verbose=True,word_timestamps=True)
pkDict = {"facePos":facePos,"faceFrames":faceFrames,"preds":predArray,"transcription":transcription}
with open("outputs/npz/"+videoName+'.pkl', 'wb') as f:  # open a text file
    pickle.dump(pkDict, f) # serialize the list

[00:00.320 --> 00:05.500]  y la población civil de guta oriental en siria sigue sufriendo los efectos de la guerra.


c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [ ]:
# EXTRAER PARTE APROPIADA DE LA TRANSCRIPCIÓN
wordArr = []
alignArr = []
for seg in transcription["segments"]:
    for w in seg["words"]:
        wordArr.append(w["word"])
        alignArr.append((w["start"],w["end"]))
print(wordArr)
print(alignArr)
pdRows = []

for speakerN in totalScores.keys():
    for (ini,end) in getSpeaking(predArray[speakerN],5,25):
        iniW, endW = -1, -1
        for i, (alignIni, alignEnd) in enumerate(alignArr):
            if iniW == -1 and ini <= alignEnd and ini >= alignIni:
                iniW = i
            if endW == -1 and end <= alignEnd and end >= alignIni:
                endW = i
        if iniW > -1 and endW > -1:
            newRow = {'video':inputVideo, 'speaker':speakerN, 'ini': ini, 'end':end, 'dataPath': "outputs/npz/"+videoName+".pkl", 'transcription':''.join(wordArr[iniW:endW+1])}
            pdRows.append(newRow)

df = pd.DataFrame(pdRows,columns=['video', 'speaker', 'ini', 'end', 'dataPath', 'transcription'])
df.to_csv(r"outputs/res.csv")

[' y', ' la', ' población', ' civil', ' de', ' guta', ' oriental', ' en', ' siria', ' sigue', ' sufriendo', ' los', ' efectos', ' de', ' la', ' guerra.']
[(0.32, 0.84), (0.84, 0.94), (0.94, 1.2), (1.2, 1.62), (1.62, 1.86), (1.86, 2.1), (2.1, 2.56), (2.56, 2.66), (2.66, 3.14), (3.14, 3.54), (3.54, 4.22), (4.22, 4.48), (4.48, 5.04), (5.04, 5.1), (5.1, 5.24), (5.24, 5.5)]


In [35]:
with open("outputs/npz/"+videoName+'.pkl', 'rb') as f:  # open a text file
    p = pickle.load(f) # serialize the list
print(p["facePos"][0])

[(215, 75, 280, 162), (215, 75, 280, 162), (214, 75, 279, 161), (213, 73, 278, 161), (212, 72, 278, 160), (211, 72, 277, 160), (211, 72, 277, 160), (211, 73, 276, 160), (210, 72, 275, 159), (209, 73, 275, 159), (209, 72, 275, 158), (208, 72, 275, 158), (208, 72, 275, 158), (208, 72, 274, 158), (208, 72, 274, 158), (208, 72, 274, 157), (208, 71, 274, 156), (208, 71, 274, 155), (208, 70, 274, 155), (208, 70, 274, 154), (208, 70, 275, 154), (209, 70, 275, 154), (210, 69, 276, 153), (211, 69, 276, 154), (211, 70, 277, 155), (212, 71, 278, 155), (213, 72, 279, 155), (214, 72, 280, 155), (215, 72, 281, 156), (216, 72, 283, 156), (217, 70, 284, 157), (218, 69, 285, 157), (220, 69, 287, 157), (221, 69, 288, 157), (222, 70, 289, 157), (223, 70, 290, 157), (224, 70, 291, 158), (224, 71, 291, 158), (225, 71, 292, 158), (226, 71, 293, 158), (227, 71, 294, 158), (228, 71, 295, 159), (228, 71, 295, 158), (229, 72, 295, 158), (229, 71, 296, 158), (229, 71, 296, 158), (229, 70, 296, 158), (229, 71, 29